In [ ]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [365]:
#Settings for accessing Website

executable_path = {"executable_path": "chromedriver.exe"}

browser = Browser ('chrome', **executable_path, headless=False)


In [ ]:
#Create empty df
NCAA_refer = "sports-reference"
ncaa_df = pd.DataFrame()

In [210]:
url = "https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min=1993&year_max=2016&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&games_type=A&c1stat=fga&c1comp=gt&order_by=year_id&order_by_asc=Y"

In [366]:
browser.visit(url)

'Next page'

In [367]:
#Clicking to the next page


def scrape_ncaa_data(page_url):
    global ncaa_df
    url = page_url
    browser.visit(url)
    click_count = 0
    
    for x in range (3):
        try:
            if click_count == 0:

                #Get html of web page
                html = browser.html

                #Read html using pandas
                read_html = pd.read_html(html)

                #Create DF and append to ncaa_df
                page_df = pd.DataFrame(read_html[1])
                ncaa_df = page_df

                #Add to click counter
                click_count += 1

                #click to go to next page
                next_page = browser.find_link_by_text("Next page")
                next_page.click()


            else:
                #Get html of web page
                html = browser.html

                #Read html using pandas
                read_html = pd.read_html(html)

                #print(read_html[1])
                page_df = pd.DataFrame(read_html[1])
                ncaa_df = ncaa_df.append(page_df, ignore_index = True)

                #click to go to next page
                next_page = browser.find_link_by_text("Next page")
                next_page.click()
        except AttributeError as error:
            print("End of scrape")
            window.close
            



In [368]:
scrape_ncaa_data(url)

In [369]:
ncaa_df.head()

list((ncaa_df.columns.values))

[('Unnamed: 0_level_0', 'Rk'),
 ('Totals', 'Player'),
 ('Unnamed: 2_level_0', 'Class'),
 ('Unnamed: 3_level_0', 'Season'),
 ('Unnamed: 4_level_0', 'Pos'),
 ('Unnamed: 5_level_0', 'School'),
 ('Unnamed: 6_level_0', 'Conf'),
 ('Unnamed: 7_level_0', 'G'),
 ('Unnamed: 8_level_0', 'MP'),
 ('Unnamed: 9_level_0', 'FG'),
 ('Unnamed: 10_level_0', 'FGA'),
 ('Unnamed: 11_level_0', '2P'),
 ('Unnamed: 12_level_0', '2PA'),
 ('Unnamed: 13_level_0', '3P'),
 ('Unnamed: 14_level_0', '3PA'),
 ('Unnamed: 15_level_0', 'FT'),
 ('Unnamed: 16_level_0', 'FTA'),
 ('Unnamed: 17_level_0', 'ORB'),
 ('Unnamed: 18_level_0', 'DRB'),
 ('Unnamed: 19_level_0', 'TRB'),
 ('Unnamed: 20_level_0', 'AST'),
 ('Unnamed: 21_level_0', 'STL'),
 ('Unnamed: 22_level_0', 'BLK'),
 ('Unnamed: 23_level_0', 'TOV'),
 ('Unnamed: 24_level_0', 'PF'),
 ('Unnamed: 25_level_0', 'PTS')]

In [370]:
# #Resetting column names
ncaa_cols =["Rk", "Player", "Class", "Season", "Pos", "School", "Conf", "G", "MP", "FG", "FGA", "2P", "2PA", "3P",\
            "3PA", "FT", "FTA", "ORB", "DRB", "TRB", "AST", "STL","BLK", "TOV", "PF", "PTS"]

ncaa_df.columns = ncaa_cols
list(ncaa_df.columns.values)
ncaa_df.head()





,Rk,Player,Class,Season,Pos,School,Conf,G,MP,FG,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
1,2,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
2,3,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
3,4,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0
4,5,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,NaN,34,...,33,NaN,NaN,29,50,19,3,NaN,NaN,114


In [371]:
#Delete Rk column
ncaa_df = ncaa_df.drop("Rk", axis =1)

# ncaa_df = ncaa_df.drop("Rk", axis =1)
ncaa_df.head()




,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,16,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
1,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,7,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
2,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,75,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
3,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0
4,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,NaN,34,95,...,33,NaN,NaN,29,50,19,3,NaN,NaN,114


In [373]:
#Define a function to apply to Season's column of DataFrame

def normalize_year(y):
    if y == "NaN":
        return y
        
    else:
        x = y
        x = y[0:3] + x[-1]

    return x

normalize_year(ncaa_df["Season"][0])

'1993'

In [402]:
# Delete repeating columns
ncaa_df = ncaa_df[(ncaa_df.Season != "Season")]
ncaa_df = ncaa_df[(ncaa_df.Season != "NaN")]
ncaa_df[ncaa_df.Season == "NaN"]

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS


In [395]:
#Change Years into a single year value

ncaa_df.head()

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,NaN,6,16,...,12,NaN,NaN,22,14,1,0,NaN,NaN,24
1,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,NaN,0,7,...,2,NaN,NaN,2,0,0,0,NaN,NaN,0
2,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,75,...,62,NaN,NaN,55,181,30,6,NaN,NaN,69
3,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,NaN,0,0,...,0,NaN,NaN,0,2,1,0,NaN,NaN,0
4,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,NaN,34,95,...,33,NaN,NaN,29,50,19,3,NaN,NaN,114


In [264]:
#Replace NaN values with 0

ncaa_df = ncaa_df.fillna(0)
ncaa_df.head()

,Player,Class,Season,Pos,School,Conf,G,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Craig Aamot,SO,1992-93,G,Marquette,GMWC,20,0,6,16,...,12,0,0,22,14,1,0,0,0,24
1,Kahhar Abdul-Wali,FR,1992-93,G,Coppin State,MEAC,9,0,0,7,...,2,0,0,2,0,0,0,0,0,0
2,Abdul Abdullah,JR,1992-93,G,Providence,Big East,32,822,18,75,...,62,0,0,55,181,30,6,0,0,69
3,Shawn Abercrombie,SO,1992-93,G,St. Bonaventure,A-10,1,0,0,0,...,0,0,0,0,2,1,0,0,0,0
4,Frank Ableson,JR,1992-93,G,Cornell,Ivy,26,0,34,95,...,33,0,0,29,50,19,3,0,0,114
